## Few-Shot-Learning


### Divisão dos Dados

Cria um dicionário dos dados (class map): {label_classe: [lista de  imagens dessa classe]} e divide os dados entre treino e teste

**Estrutura do Omniglot**


omniglot-py/ <br>
└── images_background/  (ou images_evaluation/)<br>
    ├── Alphabet_of_the_Magi/<br>
    │   ├── character01/<br>
    │   │   ├── 0709_01.png<br>
    │   │   ├── 0709_02.png<br>
    │   │   └── ... (20 imagens no total)<br>
    │   └── character02/<br>
    │       └── ...<br>
    ├── Angelic/<br>
    │   └── ...<br>
    └── ... (mais 30 alfabetos)<br>

In [2]:
import torchvision as tv
from torchvision import transforms


transform = transforms.Compose([transforms.ToTensor()])

omniglot_train = tv.datasets.Omniglot(
    root="../data",
    background=True, #Conjunto de treino
    download=True,
    transform=transform 
)


omniglot_test = tv.datasets.Omniglot(
    root="../data",
    background=False, #Conjunto de teste
    download=True,
    transform=transform
)

100.0%
100.0%


O Omniglot é dividido em background e evaluation. O conjunto background contém os alfabetos para treinar o seu modelo a "aprender a aprender". O conjunto evaluation contém alfabetos completamente novos, que o modelo nunca viu, e é usado para testar se ele consegue generalizar para novas classes com poucos exemplos.

### Episódios

Em vez de treinar o modelo para ser um especialista em um conjunto fixo de classes, nós o treinamos para ser um generalista em aprender.

Seria dar ao modelo uma série de "mini testes surpresa" (os episódios). Cada teste é uma pequena tarefa de classificação.

**Anatomia de um Episódio**

Em um cenário N-way K-shot (onde N = 5 e K = 1):

1. Amostragem: Primeiro, escolhemos aleatoriamente N classes do nosso enorme conjunto de dados de treinamento. 
2. Support Set: Para cada uma dessas N classes, damos ao modelo K exemplos. Este é o "material de estudo" para este teste específico. O modelo deve olhar para essas "N times K" imagens e aprender a distinguir as N classes.
3. Query Set: Em seguida, pegamos outras imagens (que não estavam no material de estudo) daquelas mesmas N classes. Estas são as "perguntas do teste".
4. Avaliação e Aprendizado: O modelo tenta classificar as imagens do Query Set. Calculamos o quão bem ele se saiu (loss). Usamos essa nota para ajustar os pesos do modelo através do backpropagation.


Ao repetir esse processo milhares de vezes, com milhares de combinações diferentes de classes, o modelo não está memorizando "o que é o caractere A do alfabeto Grego". Em vez disso, ele está aprendendo uma estratégia para, dado qualquer conjunto de N novas classes com K exemplos cada, descobrir as características que as diferenciam.

In [3]:
import torch, random
import numpy as np


def create_episode(classes, n_way, k_shot, k_query):
    episode_classes_idx = random.sample(list(classes.keys()), n_way)
    
    support_set = []
    support_labels = []
    query_set = []
    query_labels = []
    
    
    #* Montar o Support e o Query set para cada classe selecionada
    for i, class_idx in enumerate(episode_classes_idx):
        images_of_class = classes[class_idx] 
        
        #* Sortear imagens dessa classe
        selected_images = random.sample(images_of_class, k_shot + k_query)
        
        #* Dividir as imagens sorteadas nos dois conjuntos
        support_images = selected_images[:k_shot]
        query_images = selected_images[k_shot:]
        
        support_set.extend(support_images)
        query_set.extend(query_images)
        
        #* Criar as labels RELATIVAS ao episódio
        support_labels.extend([i] * k_shot)
        query_labels.extend([i] * k_query)

        #* Embaralhar os conjuntos
        s_indices = np.random.permutation(len(support_set))
        q_indices = np.random.permutation(len(query_set))
        
        #* Reordenar usando os índices embaralhados
        support_set = torch.stack(support_set)[s_indices]
        support_labels = torch.tensor(support_labels)[s_indices]
        query_set = torch.stack(query_set)[q_indices]
        query_labels = torch(query_labels)[q_indices]
        
        #*Retorna os 4 tensores para serem usados pelo modelo
        return support_set, support_labels, query_set, query_labels
    


### Metric Based Model

A ideia central é: aprender um espaço de características (embedding space) onde imagens da mesma classe fiquem muito próximas e imagens de classes diferentes fiquem distantes.

In [ ]:
transform = transforms.Compose([
    
])